In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
# cuda = cuda.is_available()
# print(f'Train on gpu: {cuda}')
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:


# Define transformation to apply on input images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:
# Load Caltech101 dataset using ImageFolder
dataset = ImageFolder(root='/kaggle/input/multiclass-weather-dataset/Multi-class Weather Dataset', transform=transform)

# Split dataset into train and test sets
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

# Define data loaders to load data in batches
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [6]:
alexnet = torchvision.models.alexnet(pretrained=True)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 160MB/s]  


In [7]:
# Extract features using AlexNet
train_features = []
train_targets = []
for inputs, labels in train_loader:
    with torch.no_grad():
        alexnet.eval()
        feat = alexnet.features(inputs).view(inputs.size(0), -1)
        train_features.append(feat.numpy())
        train_targets.append(labels.numpy())
train_features = np.concatenate(train_features, axis=0)
train_targets = np.concatenate(train_targets, axis=0)

In [8]:
test_features = []
test_targets = []
for inputs, labels in test_loader:
    with torch.no_grad():
        alexnet.eval()
        feat = alexnet.features(inputs).view(inputs.size(0), -1)
        test_features.append(feat.numpy())
        test_targets.append(labels.numpy())

test_features = np.concatenate(test_features, axis=0)
test_targets = np.concatenate(test_targets, axis=0)

In [9]:
train_features[0].shape

(9216,)

In [10]:
train_features, train_targets

(array([[0.        , 0.        , 0.        , ..., 2.6994212 , 3.9409413 ,
         5.44992   ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.0493542 ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [2.3106234 , 0.00731675, 0.18183061, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]], dtype=float32),
 array([3, 3, 0, 3, 0, 3, 0, 3, 1, 0, 3, 1, 1, 2, 0, 0, 3, 1, 1, 3, 0, 1,
        0, 3, 2, 3, 0, 3, 2, 0, 0, 3, 3, 0, 2, 1, 0, 3, 1, 3, 2, 2, 1, 3,
        1, 3, 0, 0, 3, 2, 3, 2, 3, 2, 2, 1, 0, 0, 2, 0, 3, 3, 2, 1, 0, 3,
        0, 1, 0, 3, 2, 1, 0, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 1, 2, 3, 3, 1,
        3, 1, 2, 3, 0, 2, 1, 1, 0, 1, 0, 3, 0, 1, 0, 1, 2, 3, 2, 1, 3, 0,
        1, 3, 2, 2, 2, 3

In [11]:
from sklearn.linear_model import LogisticRegression
lrmodel = LogisticRegression(solver = 'saga', max_iter = 100)
lrmodel.fit(train_features, train_targets)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(solver='saga')

In [12]:
y_pred = lrmodel.predict(test_features)
acc = accuracy_score(y_pred, test_targets)
print(acc)

0.96


In [13]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB()
gnb.fit(train_features, train_targets)

GaussianNB()

In [14]:
y_pred = gnb.predict(test_features)
acc = accuracy_score(y_pred, test_targets)
print(acc)

0.6755555555555556


In [15]:
from sklearn.svm import LinearSVC
clf = LinearSVC(max_iter = 5000)
clf.fit(train_features, train_targets)

LinearSVC(max_iter=5000)

In [16]:
y_pred = clf.predict(test_features)
acc = accuracy_score(y_pred, test_targets)
print(acc)

0.9688888888888889


In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(train_features, train_targets)

KNeighborsClassifier()

In [18]:
y_pred = knn.predict(test_features)
acc = accuracy_score(y_pred, test_targets)
print(acc)

0.8444444444444444
